In [54]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json

df = pd.read_csv("../requests.csv") # .query('type == "step"')

import re
def get_url(request_json):
    url = re.search(r"url='([^']*)'", request_json)
    if url is None:
        return None
    return url.group(1)



# df['url'] = df.input.apply(get_url)
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,created_at,type,input,error,session_id,id,output,duration_ms
0,2025-01-17 12:16:22.473486+00,observe,{'session': NotteSession(browser=<notte.browse...,NaN,b42383d3-4f54-436b-9372-02a81d0205eb,5d9f35ce-c8a8-49b9-a318-f311fb176511,session=SessionResponse(session_id='b42383d3-4...,5971
1,2025-01-17 12:16:13.023289+00,scrape,{'session': NotteSession(browser=<notte.browse...,NaN,5ec30427-71a2-45bb-8ad3-5af5fe8aabad,9813cdb8-b58c-461e-a480-f2c516d39531,session=SessionResponse(session_id='5ec30427-7...,6458
2,2025-01-17 12:15:02.553197+00,scrape,{'session': NotteSession(browser=<notte.browse...,NaN,3c3115c2-3d43-43b7-b89f-1ea9f1c1d21b,5323ad71-3d39-45a2-8680-5a2e5540cbe1,session=SessionResponse(session_id='3c3115c2-3...,6330
3,2025-01-17 11:12:34.147438+00,step,{'session': NotteSession(browser=<notte.browse...,500: Execution of action 'B21' failed,6711a98c-8708-4600-81cf-315ea084234d,e938a9fa-a399-44cd-adc2-0eb91e2c6a2c,"no output, check error",5182
4,2025-01-17 11:12:20.082439+00,step,{'session': NotteSession(browser=<notte.browse...,500: Multiple dialogs found in {'role': 'WebAr...,6711a98c-8708-4600-81cf-315ea084234d,501fe003-5365-4f49-8f2b-418da4fb5199,"no output, check error",15126


In [25]:
df.error.str.contains("500: Execution of action").sum()

np.int64(0)

In [70]:
# group by session_id and store the results as trajectories
trajectories = df.groupby("session_id").apply(lambda x: pd.Series({'trajectory': x.sort_values("created_at"), 'error': x.error.dropna().values}))
trajectories['error'] = trajectories['error'].apply(lambda x: x[0] if len(x) else '')
trajectories['success'] = trajectories['error'].str.len() == 0
trajectories['steps'] = trajectories['trajectory'].apply(len)
trajectories['url'] = trajectories['trajectory'].apply(lambda x: get_url(x.iloc[0].input))
trajectories['type'] = trajectories['trajectory'].apply(lambda x: x.iloc[0].type)
step_trajs = trajectories.query('not success and steps > 1')
step_trajs


/var/folders/b8/2jd9x_r53sn9l_wjrtwg_0080000gn/T/ipykernel_98670/3740524935.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trajectories = df.groupby("session_id").apply(lambda x: pd.Series({'trajectory': x.sort_values("created_at"), 'error': x.error.dropna().values}))


,trajectory,error,success,steps,url,type
session_id,,,,,,
076e973b-8fdc-4071-bab2-4bc3599e79b6,created_at type \ ...,500: Execution of action 'B17' failed,False,2,https://www.mexc.co/fr-FR,observe
3d4455a1-abc3-4325-b815-52b7e45f0288,created_at type \ ...,"500: Accessibility.snapshot: Target page, cont...",False,5,https://www.allrecipes.com,observe
43c67833-bd5a-43f6-8b33-1ccf466dd6e8,created_at type \ ...,500: Execution of action 'L5' failed,False,2,https://www.notte.cc,observe
6711a98c-8708-4600-81cf-315ea084234d,created_at type \ 5...,500: Execution of action 'B21' failed,False,3,https://www.allrecipes.com,observe
a5d31ec2-b123-4316-ae41-3dd7d1b10cbe,created_at type \ ...,500: Description should not be empty for a ful...,False,5,https://www.binance.com/fr,observe


In [ ]:
import chevron 
prompt_template = """
You are an expert in the playwright python library. 
You are given a html code of an element that failed to be interacted with
and some information about the action that was executed.
along with the error message encountered.

1. I want you to provide me with a report of why the current action failed to be executed.
Put your report inside <failure_report> tags, i.e
<failure_report>
...
</failure_report>

2. Then provide me with some suggestions on how to fix the action.
Put your suggestions inside <failure_suggestions> tags, i.e
<failure_suggestions>
...
</failure_suggestions>

3. If you believe you can generate some code to fix the action execution, provide it inside <playwright_retry_code> tags, i.e
<playwright_retry_code>
...
</playwright_retry_code>

4. If you believe, there is nothing that can be done to fix the action execution, provide a short 1-2 sentence high level explanation of why the action failed to be executed 
inside <failure_explanation> tags, i.e
<failure_explanation>
...
</failure_explanation>

Here is some contextual information to help you solve your task:

<info_failed_action>
{{info_failed_action}}
</info_failed_action>

<info_failed_action_playwright_code>
{{info_failed_action_playwright_code}}
</info_failed_action_playwright_code>

<info_error_message>
{{info_error_message}}
</info_error_message>

<info_metadata_about_html_element>
{{info_metadata_about_html_element}}
</info_metadata_about_html_element>

Please proceed with your task:
"""


chevron.render(
            prompt_template,
            {
                "info_failed_action": "click",
                "info_failed_action_playwright_code": "locator.click()",
                "info_error_message": "Element not found",
                "info_metadata_about_html_element": "locator = page.locator('internal:role=button[name="Close"i]')",
            }
)

            
            